In [1]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer,BertForSequenceClassification
import pytorch_lightning as pl

#日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,num_labels=2
)
bert_sc = bert_sc.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
label_list = [1,0,1]

#データの符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()

#推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits#分類スコア
labels_predicted = scores.argmax(-1)#スコアが最も値ラベル
num_correct = (labels_predicted==labels).sum().item()#正解数
accuracy = num_correct/labels.size(0)#精度

print("# scoresのサイズ:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

# scoresのサイズ:
torch.Size([3, 2])
# predicted labels:
tensor([0, 0, 0], device='cuda:0')
# accuracy:
0.3333333333333333
